# Generate minimals (-1)

In [ ]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pickle
from scipy.io import loadmat
import os
import matplotlib.cm as cm


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device to train")

mycase="find_minimals"
casenum="d4_m2_k005"


all_path="/content/drive/MyDrive/Colab_Notebooks/2023_MEP/0408/"
model_Parameters_name = all_path+casenum+"/"+"model_MEP_"+mycase[0:2]+"_"+"pre2"+"pretrain"+".pth" # Path

model_name=all_path+casenum+"/"+"model_MEP_"+mycase+"_"+casenum+".pth" # Path


alpha1=10000
alpha5=1
kappa=0.05


load_model = False
batches = 20000

learning_rate = 1e-4

dn=31
x_dim=4

pre_train_step=8000


def mkdir_pre(path):
  folder = os.path.exists(path)
  if not folder:
    os.makedirs(path)
    print("---  new folder...  ---")
    print("---  OK  ---")
  else:
    print("---  There is this folder!  ---")


def fig_loss_batch_pre(plt_batch,loss_batch):
    plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,loss_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("Loss")

def fig_cos_pre(plt_batch,lg_batch,j):
    fig = plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,lg_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("loss on grid")
    if j == batches-500:
      plt.savefig(all_path+casenum+"/"+"lgloss"+".jpg")


def fig_countour_pre(model,j):
    plot_use=[0,0.25,0.5,0.75,1]

    x1 = np.linspace(0.001,0.999,dn,endpoint=True)
    x2 = np.linspace(0.001,0.999,dn,endpoint=True)

    xx1,xx2=np.meshgrid(x1,x2)
    xx1=xx1.reshape((-1,1))
    xx2=xx2.reshape((-1,1))

    xx=np.hstack((xx1,xx2))

    xx1=xx[:,0]
    xx2=xx[:,1]

    xx1=xx1.reshape(dn,dn)
    xx2=xx2.reshape(dn,dn)

    fig2 = plt.figure(figsize=(16,16))

    for k in range(len(plot_use)):
      for l in range(len(plot_use)):
        x2=plot_use[k]*np.ones((dn*dn,1))
        x3=plot_use[l]*np.ones((dn*dn,1))
        xxx=np.hstack((xx,x2))
        xxxx=np.hstack((xxx,x3))
        xxxx_tensor=Variable(torch.from_numpy(xxxx),requires_grad=True).to(device)
        phi_tensor=model(xxxx_tensor)

        phi=phi_tensor.cpu().detach().numpy()

        phi_plot_sq=phi.reshape(dn,dn)


        fig2.add_subplot(len(plot_use),len(plot_use),k*len(plot_use)+l+1)

        picori=phi_plot_sq.reshape(dn,dn)
        pic=np.zeros((dn,dn,3))

        B=(np.sign(picori)+1)/2*picori
        C=(np.sign(picori)-1)/2*picori

        pic[:,:,0]=B
        pic[:,:,-1]=C


        plt.imshow(pic)


    plt.savefig(all_path+casenum+"/"+str(j)+".jpg")



class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(4,64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, s):
        phi_pred = self.linear_tanh_stack(s)
        return phi_pred



def train(model):
    loss_batch=[]
    plt_batch=[]
    lg_batch=[]

    for i in range(batches):
        if i<2000:
            learning_rate=1e-3
        elif i<15000:
            learning_rate=1e-4
        else:
            learning_rate=1e-5

        if i<2000:
          alpha1=10
          alpha5=10
        else:
          alpha1=10
          alpha5=1

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


        datanum=500
        x_np = np.random.uniform(0.001,0.999,(datanum,4))


        x_tensor=Variable(torch.from_numpy(x_np),requires_grad=True).to(device)

        x_all=[]
        for j in range(x_dim):
          x_all.append(x_tensor[:,j:j+1])

        x_tensor_all=torch.cat(x_all,1)
        phi_int_tensor=model(x_tensor_all)

        bound_samp=100
        x00=np.random.uniform(0.001,0.999,(bound_samp,4))
        x00[:,0]=0.0
        x00_tensor=Variable(torch.from_numpy(x00),requires_grad=True).to(device)

        x01=np.random.uniform(0.001,0.999,(bound_samp,4))
        x01[:,0]=1.0
        x01_tensor=Variable(torch.from_numpy(x01),requires_grad=True).to(device)

        x10=np.random.uniform(0.001,0.999,(bound_samp,4))
        x10[:,1]=0.0
        x10_tensor=Variable(torch.from_numpy(x10),requires_grad=True).to(device)

        x11=np.random.uniform(0.001,0.999,(bound_samp,4))
        x11[:,1]=1.0
        x11_tensor=Variable(torch.from_numpy(x11),requires_grad=True).to(device)

        x20=np.random.uniform(0.001,0.999,(bound_samp,4))
        x20[:,2]=0.0
        x20_tensor=Variable(torch.from_numpy(x20),requires_grad=True).to(device)

        x21=np.random.uniform(0.001,0.999,(bound_samp,4))
        x21[:,2]=1.0
        x21_tensor=Variable(torch.from_numpy(x21),requires_grad=True).to(device)

        x30=np.random.uniform(0.001,0.999,(bound_samp,4))
        x30[:,3]=0.0
        x30_tensor=Variable(torch.from_numpy(x30),requires_grad=True).to(device)

        x31=np.random.uniform(0.001,0.999,(bound_samp,4))
        x31[:,3]=1.0
        x31_tensor=Variable(torch.from_numpy(x31),requires_grad=True).to(device)

        phi_00=model(x00_tensor)
        phi_01=model(x01_tensor)

        phi_10=model(x10_tensor)
        phi_11=model(x11_tensor)

        phi_20=model(x20_tensor)
        phi_21=model(x21_tensor)

        phi_30=model(x30_tensor)
        phi_31=model(x31_tensor)

        phi_p1_np=1.0*np.ones((bound_samp))
        phi_m1_np=-1.0*np.ones((bound_samp))

        phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
        phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

        loss_bound=torch.mean((phi_00.reshape(-1)-phi_p1)**2)+torch.mean((phi_01.reshape(-1)-phi_p1)**2)+\
        torch.mean((phi_10.reshape(-1)-phi_m1)**2)+torch.mean((phi_11.reshape(-1)-phi_m1)**2)+\
        torch.mean((phi_20.reshape(-1)-phi_p1)**2)+torch.mean((phi_21.reshape(-1)-phi_p1)**2)+\
        torch.mean((phi_30.reshape(-1)-phi_m1)**2)+torch.mean((phi_31.reshape(-1)-phi_m1)**2)

        if i<pre_train_step:
          loss_res=torch.mean((phi_int_tensor+1)**2)
          loss=alpha1*loss_res+alpha5*loss_bound

        else:
          gradx_all=[]
          for j in range(x_dim):
            gradx_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx_all.append(gradx_int)

          gradx2_int=0.0
          for j in range(x_dim):
            gradx2_j=torch.autograd.grad(outputs=gradx_all[j],inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx2_int=gradx2_int+gradx2_j

          loss_res=torch.mean((kappa**2*gradx2_int-(phi_int_tensor**3-phi_int_tensor))**2)

          loss = alpha1*loss_res +alpha5*loss_bound


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i) % 50 == 0:
            test_sample=10000

            xx=np.random.uniform(0.001,0.999,(test_sample,4))

            xx_tensor=Variable(torch.from_numpy(xx),requires_grad=True).to(device)

            xx_all=[]
            for j in range(x_dim):
              xx_all.append(xx_tensor[:,j:j+1])

            xx_int_tensor=torch.cat(xx_all,1)

            phi=model(xx_int_tensor)

            gradxx_all=[]
            abs_gradxx_phi_2=0.0
            for j in range(x_dim):
              gradxx_int = torch.autograd.grad(outputs=phi,inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx_all.append(gradxx_int)


            gradxx2_int=0.0
            for j in range(x_dim):
              gradxx2_j=torch.autograd.grad(outputs=gradxx_all[j],inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx2_int=gradxx2_int+gradxx2_j


            L_phi=(kappa**2*gradxx2_int-(phi**3-phi))**2
            L_int=torch.mean(L_phi,0)

            lg_batch.append(L_int.item())

            if i<pre_train_step:
              loss,batch=loss.item(),i
              print("iteration=",i,"loss=", loss)
            else:
              loss, batch = alpha1*loss_res.item()+alpha5*loss_bound.item(), i
              print(f'batches: {batch+1}')
              print(f'loss1: {loss_res} loss_bound: {loss_bound}')

            loss_batch.append(loss)
            plt_batch.append(i+1)
            plt.figure()

        if (i) % 500 == 0:

            fig_cos_pre(plt_batch,lg_batch,i)
            fig_loss_batch_pre(plt_batch,loss_batch)
            fig_countour_pre(model,i)
            plt.show()


            torch.save(model.state_dict(), model_name)
            print("Saved PyTorch Model State to " +str(model_name))


    return




if __name__=='__main__':

    mkdir_pre(all_path+casenum+"/")
    print(all_path+casenum+"/")


    model = NeuralNetwork().to(device)
    model = model.double()

    train(model)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Generate minimals (1)

In [ ]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pickle
from scipy.io import loadmat
import os
import matplotlib.cm as cm


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device to train")

mycase="find_minimals"
casenum="d4_p1_k005"


all_path="/content/drive/MyDrive/Colab_Notebooks/2023_MEP/0408"
model_Parameters_name = all_path+casenum+"/"+"model_MEP_"+mycase[0:2]+"_"+"pre2"+"pretrain"+".pth" # Path

model_name=all_path+casenum+"/"+"model_MEP_"+mycase+"_"+casenum+".pth" # Path


alpha1=10000
alpha5=1
kappa=0.05


load_model = False
batches = 20000

learning_rate = 1e-4

dn=31
x_dim=4

pre_train_step=2000


def mkdir_pre(path):
  folder = os.path.exists(path)
  if not folder:
    os.makedirs(path)
    print("---  new folder...  ---")
    print("---  OK  ---")
  else:
    print("---  There is this folder!  ---")


def fig_loss_batch_pre(plt_batch,loss_batch):
    plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,loss_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("Loss")

def fig_cos_pre(plt_batch,lg_batch,j):
    fig = plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,lg_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("loss on grid")
    if j == batches-500:
      plt.savefig(all_path+casenum+"/"+"lgloss"+".jpg")


def fig_countour_pre(model,j):
    plot_use=[0,0.25,0.5,0.75,1]

    x1 = np.linspace(0.001,0.999,dn,endpoint=True)
    x2 = np.linspace(0.001,0.999,dn,endpoint=True)

    xx1,xx2=np.meshgrid(x1,x2)
    xx1=xx1.reshape((-1,1))
    xx2=xx2.reshape((-1,1))

    xx=np.hstack((xx1,xx2))

    xx1=xx[:,0]
    xx2=xx[:,1]

    xx1=xx1.reshape(dn,dn)
    xx2=xx2.reshape(dn,dn)

    fig2 = plt.figure(figsize=(16,16))

    for k in range(len(plot_use)):
      for l in range(len(plot_use)):
        x2=plot_use[k]*np.ones((dn*dn,1))
        x3=plot_use[l]*np.ones((dn*dn,1))
        xxx=np.hstack((xx,x2))
        xxxx=np.hstack((xxx,x3))
        xxxx_tensor=Variable(torch.from_numpy(xxxx),requires_grad=True).to(device)
        phi_tensor=model(xxxx_tensor)

        phi=phi_tensor.cpu().detach().numpy()

        phi_plot_sq=phi.reshape(dn,dn)


        fig2.add_subplot(len(plot_use),len(plot_use),k*len(plot_use)+l+1)

        picori=phi_plot_sq.reshape(dn,dn)
        pic=np.zeros((dn,dn,3))

        B=(np.sign(picori)+1)/2*picori
        C=(np.sign(picori)-1)/2*picori

        pic[:,:,0]=B
        pic[:,:,-1]=C

        plt.imshow(pic)


    plt.savefig(all_path+casenum+"/"+str(j)+".jpg")



class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(4,64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, s):
        phi_pred = self.linear_tanh_stack(s)
        return phi_pred



def train(model):
    loss_batch=[]
    plt_batch=[]
    lg_batch=[]

    for i in range(batches):

        if i<2000:
            learning_rate=1e-3
        elif i<15000:
            learning_rate=1e-4
        else:
            learning_rate=1e-5

        if i<2000:
          alpha1=10
          alpha5=10
        else:
          alpha1=10
          alpha5=1

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


        datanum=500
        x_np = np.random.uniform(0.001,0.999,(datanum,4))


        x_tensor=Variable(torch.from_numpy(x_np),requires_grad=True).to(device)

        x_all=[]
        for j in range(x_dim):
          x_all.append(x_tensor[:,j:j+1])

        x_tensor_all=torch.cat(x_all,1)
        phi_int_tensor=model(x_tensor_all)

        bound_samp=100
        x00=np.random.uniform(0.001,0.999,(bound_samp,4))
        x00[:,0]=0.0
        x00_tensor=Variable(torch.from_numpy(x00),requires_grad=True).to(device)

        x01=np.random.uniform(0.001,0.999,(bound_samp,4))
        x01[:,0]=1.0
        x01_tensor=Variable(torch.from_numpy(x01),requires_grad=True).to(device)

        x10=np.random.uniform(0.001,0.999,(bound_samp,4))
        x10[:,1]=0.0
        x10_tensor=Variable(torch.from_numpy(x10),requires_grad=True).to(device)

        x11=np.random.uniform(0.001,0.999,(bound_samp,4))
        x11[:,1]=1.0
        x11_tensor=Variable(torch.from_numpy(x11),requires_grad=True).to(device)

        x20=np.random.uniform(0.001,0.999,(bound_samp,4))
        x20[:,2]=0.0
        x20_tensor=Variable(torch.from_numpy(x20),requires_grad=True).to(device)

        x21=np.random.uniform(0.001,0.999,(bound_samp,4))
        x21[:,2]=1.0
        x21_tensor=Variable(torch.from_numpy(x21),requires_grad=True).to(device)

        x30=np.random.uniform(0.001,0.999,(bound_samp,4))
        x30[:,3]=0.0
        x30_tensor=Variable(torch.from_numpy(x30),requires_grad=True).to(device)

        x31=np.random.uniform(0.001,0.999,(bound_samp,4))
        x31[:,3]=1.0
        x31_tensor=Variable(torch.from_numpy(x31),requires_grad=True).to(device)

        phi_00=model(x00_tensor)
        phi_01=model(x01_tensor)

        phi_10=model(x10_tensor)
        phi_11=model(x11_tensor)

        phi_20=model(x20_tensor)
        phi_21=model(x21_tensor)

        phi_30=model(x30_tensor)
        phi_31=model(x31_tensor)

        phi_p1_np=1.0*np.ones((bound_samp))
        phi_m1_np=-1.0*np.ones((bound_samp))

        phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
        phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

        loss_bound=torch.mean((phi_00.reshape(-1)-phi_p1)**2)+torch.mean((phi_01.reshape(-1)-phi_p1)**2)+\
        torch.mean((phi_10.reshape(-1)-phi_m1)**2)+torch.mean((phi_11.reshape(-1)-phi_m1)**2)+\
        torch.mean((phi_20.reshape(-1)-phi_p1)**2)+torch.mean((phi_21.reshape(-1)-phi_p1)**2)+\
        torch.mean((phi_30.reshape(-1)-phi_m1)**2)+torch.mean((phi_31.reshape(-1)-phi_m1)**2)



        if i<pre_train_step:
          loss_res=torch.mean((phi_int_tensor-1)**2)
          loss=alpha1*loss_res+alpha5*loss_bound



        else:
          gradx_all=[]
          for j in range(x_dim):
            gradx_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx_all.append(gradx_int)

          gradx2_int=0.0
          for j in range(x_dim):
            gradx2_j=torch.autograd.grad(outputs=gradx_all[j],inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx2_int=gradx2_int+gradx2_j

          loss_res=torch.mean((kappa**2*gradx2_int-(phi_int_tensor**3-phi_int_tensor))**2)

          loss = alpha1*loss_res +alpha5*loss_bound


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i) % 50 == 0:
            test_sample=10000

            xx=np.random.uniform(0.001,0.999,(test_sample,4))

            xx_tensor=Variable(torch.from_numpy(xx),requires_grad=True).to(device)

            xx_all=[]
            for j in range(x_dim):
              xx_all.append(xx_tensor[:,j:j+1])

            xx_int_tensor=torch.cat(xx_all,1)

            phi=model(xx_int_tensor)

            gradxx_all=[]
            abs_gradxx_phi_2=0.0
            for j in range(x_dim):
              gradxx_int = torch.autograd.grad(outputs=phi,inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx_all.append(gradxx_int)


            gradxx2_int=0.0
            for j in range(x_dim):
              gradxx2_j=torch.autograd.grad(outputs=gradxx_all[j],inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx2_int=gradxx2_int+gradxx2_j


            L_phi=(kappa**2*gradxx2_int-(phi**3-phi))**2
            L_int=torch.mean(L_phi,0)

            lg_batch.append(L_int.item())

            if i<pre_train_step:
              loss,batch=loss.item(),i
              print("iteration=",i,"loss=", loss)
            else:
              loss, batch = alpha1*loss_res.item()+alpha5*loss_bound.item(), i
              print(f'batches: {batch+1}')
              print(f'loss1: {loss_res} loss_bound: {loss_bound}')

            loss_batch.append(loss)
            plt_batch.append(i+1)
            plt.figure()

        if (i) % 500 == 0:

            fig_cos_pre(plt_batch,lg_batch,i)
            fig_loss_batch_pre(plt_batch,loss_batch)
            fig_countour_pre(model,i)
            plt.show()


            torch.save(model.state_dict(), model_name)
            print("Saved PyTorch Model State to " +str(model_name))


    return




if __name__=='__main__':

    mkdir_pre(all_path+casenum+"/")
    print(all_path+casenum+"/")


    model = NeuralNetwork().to(device)
    model = model.double()

    #model = train_pre(model)
    train(model)



# Main

In [ ]:

import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pickle
from scipy.io import loadmat
import os
import matplotlib.cm as cm

import warnings
warnings.filterwarnings('ignore')


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device to train")


all_path="/content/drive/MyDrive/Colab_Notebooks/2023_MEP/0408/"
path_m1="/content/drive/MyDrive/Colab_Notebooks/2023_MEP/0408/d4_m2_k005/model_MEP_find_minimals_d4_m2_k005.pth"
path_p1="/content/drive/MyDrive/Colab_Notebooks/2023_MEP/0408/d4_p2_k005/model_MEP_find_minimals_d4_p2_k005.pth"


mycase="ac_4d_2"
casenum="NBC_k008_beta10lg_2"

path_lg=all_path+casenum+"/"+"lgloss"+".pkl"

plot_use_all=[]

torch.set_printoptions(precision=10)


model_name= all_path+casenum+"/"+"model_MEP_"+mycase+"_"+casenum+".pth"
model_Parameters_name=model_name


kappa=0.08
load_model = False
batches = 100000
beta = 10
learning_rate = 1e-4
dimension = 1
dn=28
x_dim=4



alpha1=1
alpha4=0
alpha3=0.001
alpha5=1



def mkdir(path):
  folder = os.path.exists(path)
  if not folder:
    os.makedirs(path)
    print("---  new folder...  ---")
    print("---  OK  ---")
  else:
    print("---  There is this folder!  ---")


def fig_loss_batch(plt_batch,loss_batch):
    plt.figure(figsize=(5,4))
    plt.plot(plt_batch,loss_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("Loss")

def fig_cos(plt_batch,lg_batch,j):
    fig = plt.figure(figsize=(5,4))
    plt.plot(plt_batch,lg_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("$l_g$")
    if j == batches-500:
      plt.savefig(all_path+casenum+"/"+"lgloss"+".jpg")


def fig_countour(model,j):
    x_plot_use=[0.25,0.5]
    t_plot_use=[0,0.25,0.5,0.75,1]

    x1 = np.linspace(0.001,0.999,dn,endpoint=True)
    x2 = np.linspace(0.001,0.999,dn,endpoint=True)

    xx1,xx2=np.meshgrid(x1,x2)
    xx1=xx1.reshape((-1,1))
    xx2=xx2.reshape((-1,1))

    xx=np.hstack((xx1,xx2))

    xx1=xx[:,0]
    xx2=xx[:,1]

    xx1=xx1.reshape(dn,dn)
    xx2=xx2.reshape(dn,dn)


    for t in t_plot_use:
      fig2 = plt.figure(figsize=(6,6))
      for k in range(len(x_plot_use)):
        for l in range(len(x_plot_use)):
          x2=x_plot_use[k]*np.ones((dn*dn,1))
          x3=x_plot_use[l]*np.ones((dn*dn,1))
          tt=t*np.ones((dn*dn,1))
          xxx=np.hstack((xx,x2))
          xxxx=np.hstack((xxx,x3))
          txxxx=np.hstack((tt,xxxx))
          txxxx_tensor=Variable(torch.from_numpy(txxxx),requires_grad=True).to(device)
          phi_tensor=model(txxxx_tensor)

          phi=phi_tensor.cpu().detach().numpy()
          phi_plot_sq=phi.reshape(dn,dn)

          fig2.add_subplot(len(x_plot_use),len(x_plot_use),k*len(x_plot_use)+l+1)

          picori=phi_plot_sq.reshape(dn,dn)
          pic=np.zeros((dn,dn,3))

          B=(np.sign(picori)+1)/2*picori
          C=(np.sign(picori)-1)/2*picori

          pic[:,:,0]=B
          pic[:,:,-1]=C

          plt.imshow(pic)


      plt.savefig(all_path+casenum+"/"+"iteration_"+str(j)+"t_"+str(t)+".jpg")

class NeuralNetwork_minimum(nn.Module):
    def __init__(self):
        super(NeuralNetwork_minimum, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(4,64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, s):
        phi_pred = self.linear_tanh_stack(s)
        return phi_pred


class NeuralNetwork(nn.Module):
    def __init__(self,p1,m1):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(x_dim+1,100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 1)
        )
        self.p1=p1
        self.m1=m1


    def forward(self, s):
        x_pred = self.linear_tanh_stack(s)
        ss=s[:,0]
        ss=ss.reshape(-1,1)
        xx=s[:,1:]

        out=ss*(1-ss)*x_pred + (1-ss)*self.p1(xx) + ss*self.m1(xx)

        return out



def train(model):
    loss_batch=[]
    plt_batch=[]
    lg_batch=[]

    if load_model:
        model.load_state_dict(torch.load(model_Parameters_name))

    for i in range(batches):

        if i<5000:
            learning_rate=1e-3
        else:
            learning_rate=1e-4

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        if i<4000:
          alpha1=10
          alpha4=1
          alpha3=0.001
          alpha5=1

        else:
          alpha1=0
          alpha4=1
          alpha3=0.001
          alpha5=0.01

        datanum=50
        s_np = np.random.uniform(0.001,0.999,(datanum,1))
        x_np = np.random.uniform(0.001,0.999,(datanum,x_dim))

        xxind=np.linspace(0,datanum-1,datanum)
        s_int_mesh_np,xxind_mesh_np=np.meshgrid(s_np,xxind)

        s_int_np=s_int_mesh_np.reshape((-1,1))
        xxind_np=xxind_mesh_np.reshape(-1)

        x_int_np=x_np[xxind_np.astype(np.int32),:]

        s_int_tensor=Variable(torch.from_numpy(s_int_np),requires_grad=True).to(device)
        x_int_tensor1=Variable(torch.from_numpy(x_int_np),requires_grad=True).to(device)

        x_all=[]
        for j in range(x_dim):
          x_all.append(x_int_tensor1[:,j:j+1])

        x_int_tensor=torch.cat(x_all,1)

        sx=torch.cat((s_int_tensor,x_int_tensor),1)

        phi_int_tensor=model(sx)

        gradx_all=[]
        abs_gradx_phi_2=0.0
        for j in range(x_dim):
          gradx_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=x_all[j],grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]
          gradx_all.append(gradx_int)
          abs_gradx_phi_2=abs_gradx_phi_2+gradx_int**2

        V_int = kappa**2/2*abs_gradx_phi_2+(phi_int_tensor**2-1)**2/4

        V_int_mesh=V_int.reshape((datanum,datanum))
        F_aint=torch.mean(V_int_mesh,0)


        grads_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=s_int_tensor,grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]
        grads2_int = torch.autograd.grad(outputs=grads_int,inputs=s_int_tensor,grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]

        gradx2_int=0.0
        for j in range(x_dim):
          gradx2_j=torch.autograd.grad(outputs=gradx_all[j],inputs=x_all[j],grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]
          gradx2_int=gradx2_int+gradx2_j


        L_phi_mesh_pre=(kappa**2*gradx2_int-(phi_int_tensor**3-phi_int_tensor))**2
        L_phi_mesh=L_phi_mesh_pre.reshape((datanum,datanum))
        L_int=torch.mean(L_phi_mesh,0)

        phi_s_mesh=grads_int.reshape((datanum,datanum))
        phi_int=torch.mean(phi_s_mesh**2,0)

        loss_1=1/beta*torch.log(torch.mean(torch.exp(beta*F_aint)))
        loss_3=torch.mean(torch.abs(grads_int*grads2_int))

        loss_4=torch.mean(torch.sqrt(L_int)*torch.sqrt(phi_int))

        bound_samp=100

        s_b = np.random.uniform(0.001,0.999,(bound_samp,1))
        s_b_tensor=Variable(torch.from_numpy(s_b),requires_grad=True).to(device)

        x00=np.random.uniform(0.001,0.999,(bound_samp,4))
        x00[:,0]=0.0
        x00_tensor=Variable(torch.from_numpy(x00),requires_grad=True).to(device)
        sx00_tensor=torch.cat((s_b_tensor,x00_tensor),1)

        x01=np.random.uniform(0.001,0.999,(bound_samp,4))
        x01[:,0]=1.0
        x01_tensor=Variable(torch.from_numpy(x01),requires_grad=True).to(device)
        sx01_tensor=torch.cat((s_b_tensor,x01_tensor),1)

        x10=np.random.uniform(0.001,0.999,(bound_samp,4))
        x10[:,1]=0.0
        x10_tensor=Variable(torch.from_numpy(x10),requires_grad=True).to(device)
        sx10_tensor=torch.cat((s_b_tensor,x10_tensor),1)


        x11=np.random.uniform(0.001,0.999,(bound_samp,4))
        x11[:,1]=1.0
        x11_tensor=Variable(torch.from_numpy(x11),requires_grad=True).to(device)
        sx11_tensor=torch.cat((s_b_tensor,x11_tensor),1)

        x20=np.random.uniform(0.001,0.999,(bound_samp,4))
        x20[:,2]=0.0
        x20_tensor=Variable(torch.from_numpy(x20),requires_grad=True).to(device)
        sx20_tensor=torch.cat((s_b_tensor,x20_tensor),1)

        x21=np.random.uniform(0.001,0.999,(bound_samp,4))
        x21[:,2]=1.0
        x21_tensor=Variable(torch.from_numpy(x21),requires_grad=True).to(device)
        sx21_tensor=torch.cat((s_b_tensor,x21_tensor),1)

        x30=np.random.uniform(0.001,0.999,(bound_samp,4))
        x30[:,3]=0.0
        x30_tensor=Variable(torch.from_numpy(x30),requires_grad=True).to(device)
        sx30_tensor=torch.cat((s_b_tensor,x30_tensor),1)

        x31=np.random.uniform(0.001,0.999,(bound_samp,4))
        x31[:,3]=1.0
        x31_tensor=Variable(torch.from_numpy(x31),requires_grad=True).to(device)
        sx31_tensor=torch.cat((s_b_tensor,x31_tensor),1)

        phi_00=model(sx00_tensor)
        phi_01=model(sx01_tensor)

        phi_10=model(sx10_tensor)
        phi_11=model(sx11_tensor)

        phi_20=model(sx20_tensor)
        phi_21=model(sx21_tensor)

        phi_30=model(sx30_tensor)
        phi_31=model(sx31_tensor)

        phi_p1_np=1.0*np.ones((bound_samp))
        phi_m1_np=-1.0*np.ones((bound_samp))

        phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
        phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

        loss_bound=torch.mean((phi_00.reshape(-1)-phi_p1)**2)+torch.mean((phi_01.reshape(-1)-phi_p1)**2)+\
        torch.mean((phi_10.reshape(-1)-phi_m1)**2)+torch.mean((phi_11.reshape(-1)-phi_m1)**2)+\
        torch.mean((phi_20.reshape(-1)-phi_p1)**2)+torch.mean((phi_21.reshape(-1)-phi_p1)**2)+\
        torch.mean((phi_30.reshape(-1)-phi_m1)**2)+torch.mean((phi_31.reshape(-1)-phi_m1)**2)


        loss = alpha1*loss_1 +alpha4*loss_4+alpha3*loss_3+alpha5*loss_bound

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i) % 50 == 0:
            ss_num=5
            xx_num=2000

            ss = np.linspace(0.001,0.999,ss_num,endpoint=True)
            xx=np.random.uniform(0.001,0.999,(xx_num,4))

            xxind=np.linspace(0,xx_num-1,xx_num)

            ss_int_mesh_np,xxind_int_mesh_np=np.meshgrid(ss,xxind)

            ss_int_np=ss_int_mesh_np.reshape((-1,1))
            xxind_np=xxind_int_mesh_np.reshape(-1)

            xx_int_np=xx[xxind_np.astype(np.int32),:]

            ss_int_tensor=Variable(torch.from_numpy(ss_int_np),requires_grad=True).to(device)
            xx_int_tensor1=Variable(torch.from_numpy(xx_int_np),requires_grad=True).to(device)

            xx_all=[]
            for j in range(x_dim):
              xx_all.append(xx_int_tensor1[:,j:j+1])

            xx_int_tensor=torch.cat(xx_all,1)

            ssxx=torch.cat((ss_int_tensor,xx_int_tensor),1)

            phi=model(ssxx)

            gradxx_all=[]
            abs_gradxx_phi_2=0.0
            for j in range(x_dim):
              gradxx_int = torch.autograd.grad(outputs=phi,inputs=xx_all[j],grad_outputs=torch.ones_like(ss_int_tensor),create_graph=True)[0]
              gradxx_all.append(gradxx_int)
              abs_gradxx_phi_2=abs_gradxx_phi_2+gradxx_int**2


            gradxx2_int=0.0
            for j in range(x_dim):
              gradxx2_j=torch.autograd.grad(outputs=gradxx_all[j],inputs=xx_all[j],grad_outputs=torch.ones_like(ss_int_tensor),create_graph=True)[0]
              gradxx2_int=gradxx2_int+gradxx2_j


            L_phi_mesh_pre=(kappa**2*gradxx2_int-(phi**3-phi))**2
            L_phi_mesh=L_phi_mesh_pre.reshape((xx_num,ss_num))
            L_int=torch.mean(L_phi_mesh,0)

            gradss_int = torch.autograd.grad(outputs=phi,inputs=ss_int_tensor,grad_outputs=torch.ones_like(ss_int_tensor),create_graph=True)[0]

            phi_ss_mesh=gradss_int.reshape((xx_num,ss_num))
            phi_int=torch.mean(phi_ss_mesh**2,0)

            lg=torch.mean(torch.sqrt(L_int)*torch.sqrt(phi_int))

            lg_batch.append(lg.item())

            loss, batch = alpha1*loss_1.item() + alpha4*loss_4.item() + alpha3*loss_3.item()+alpha5*loss_bound.item(), i
            print(f'batches: {batch+1}')
            print(f'loss1: {alpha1*loss_1} loss4: {alpha4*loss_4} loss3: {alpha3*loss_3} loss_bound: {alpha5*loss_bound}')

            loss_batch.append(loss)
            plt_batch.append(i+1)
            plt.figure()

        if (i) % 2000 == 0:

            V_int = kappa**2/2*abs_gradxx_phi_2+(phi**2-1)**2/4

            V_int_mesh=V_int.reshape((xx_num,ss_num))
            F_aint=torch.mean(V_int_mesh,0)
            energy_plot=F_aint.cpu().detach().numpy()

            fig_cos(plt_batch,lg_batch,i)
            fig_loss_batch(plt_batch,loss_batch)
            fig_countour(model,i)
            plt.show()


            torch.save(model.state_dict(), model_name)
            print("Saved PyTorch Model State to " +str(model_name))

    output = open(path_lg, 'wb')
    pickle.dump(lg_batch,output)
    output.close()

    return




if __name__=='__main__':

    net_p1=NeuralNetwork_minimum().to(device)
    net_p1=net_p1.double()
    net_p1.load_state_dict(torch.load(path_p1))

    for p in net_p1.parameters():
        p.requires_grad=False


    net_m1=NeuralNetwork_minimum().to(device)
    net_m1=net_m1.double()
    net_m1.load_state_dict(torch.load(path_m1))


    for p in net_m1.parameters():
        p.requires_grad=False


    mkdir(all_path+casenum+"/")


    model = NeuralNetwork(net_p1,net_m1).to(device)
    model = model.double()

    #model = train_pre(model)
    train(model)

